In [7]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.datasets import mnist
from keras.layers import BatchNormalization, Flatten
from keras.layers import Input, Conv2D, Convolution2D
from keras.layers import Activation, concatenate, Dropout
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D

from keras.models import Sequential, Model
from keras.utils import np_utils, to_categorical
from keras.applications.imagenet_utils import _obtain_input_shape

from tuner import utils
from tuner.tune import compute_loss

In [2]:
train_dir = 'dataset/brain/train'
validation = 'dataset/brain/validation'
df_train = utils.df_fromdir(train_dir)
df_test = utils.df_fromdir(train_dir)

x_train, y_train = utils.load_fromdf(df_train)
x_test, y_test = utils.load_fromdf(df_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [3]:
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe
from hyperas.distributions import choice, uniform, conditional

In [6]:
def cnn_block(x, 
    n_layer=2, channel=32, kernel=(3,3), padding='same' ):
    from keras import backend as K
    channel_axis = 1 if K.image_data_format()=='channels_first' else 3
    h = x
    for _ in range(n_layer):
        h = BatchNormalization()(h)
        h = Activation('relu')(h)
        h = Conv2D(channel, kernel, padding='same')(h)
        
    x = concatenate([h, x], axis=channel_axis)
    return x

In [ ]:
def block(x, 
    n_layer=2, channel=32, kernel=(3,3), padding='same' ):
    from keras import backend as K
    channel_axis = 1 if K.image_data_format()=='channels_first' else 3
    h = x
    for _ in range(n_layer):
        h = BatchNormalization()(h)
        h = Activation('relu')(h)
        h = Conv2D(channel, kernel, padding='same')(h)
        
    x = concatenate([h, x], axis=channel_axis)
    return x

In [ ]:
@compute_loss()
def lenet(n_out, input_shape=(96, 96, 3)):
    inputs = Input(shape=input_shape)
    x = inputs
    x = Conv2D(32, (3, 3))(x)
    x = block(x, n_layer=2)
    x = AveragePooling2D(pool_size=(2, 2))(x)
    x = block(x, n_layer=2)
    x = AveragePooling2D(pool_size=(2, 2))(x)
    x = Conv2D(n_out , (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax')(x)
    model = Model(inputs=inputs, outputs=x)
    return model

In [ ]:
lenet(x_train, y_train, x_test, y_test)